In [1]:
from bloqade.analog import start, get_capabilities, piecewise_linear
import math
import networkx as nx
import numpy as np
from scipy.spatial.distance import pdist, squareform
import pickle
import matplotlib.pyplot as plt

In [2]:
def MIS(graph):
    aquila_capabilities = get_capabilities()
    C6 = float(aquila_capabilities.capabilities.rydberg.c6_coefficient)
    Omega_max = float(aquila_capabilities.capabilities.rydberg.global_.rabi_frequency_max)

    Rba = np.sqrt(2*np.sqrt(2))
    Rb = Rba*4.5
    Delta_end = (C6)/Rb**6

    T_max = 4
    sweep_time = 0.8*T_max
    ramp_time = 0.1*T_max

    rabi_amplitude_values = [0.0, Omega_max, Omega_max, 0.0]
    detuning_values = [-Delta_end, -Delta_end, Delta_end, Delta_end]
    durations = [ramp_time, sweep_time, ramp_time]

    RabiProtocol = piecewise_linear(durations,rabi_amplitude_values)
    DeltaProtocol = piecewise_linear(durations,detuning_values)

    mis_udg_program=(
        graph.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
    )
    result = mis_udg_program.bloqade.python().run(100,interaction_picture=True)
    report = result.report()
    val = report.bitstrings()[0].tolist()
    return set(tuple(i) for i in val)

In [3]:
def coloring(atoms, find_colors, color, result, minres):
    if len(find_colors) == 0:
        return result
    graph = start.add_position(find_colors)
    res = MIS(graph)
    for bitstring in res:
        to_delete = []
        for (i, b) in enumerate(bitstring):
            if b:
                idx = atoms.index(find_colors[i])
                to_delete.append(atoms[idx])
                result[atoms[idx]] = color
        optres = coloring(atoms, [elem for (i, elem) in enumerate(find_colors) if elem not in to_delete], color+1, result, minres)
        if len(minres) == 0 or max(optres.values()) < max(minres.values()):
            minres = optres.copy()
    return minres